<a href="https://colab.research.google.com/github/ANUNAYBAGGA/Ai-in-Trading-Udacity/blob/master/StocksAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import os
import glob

path = 'gdrive/My Drive/stonks_DB'
extension = 'csv'
#os.chdir(path) #Use for first time only
result = glob.glob('*.{}'.format(extension))
print(result)
db = {}
for i in result:
  db[i[:-4]] = pd.read_csv(i)

print(db.keys())
stocks = list(db.keys())

['ONGC.csv', 'GAIL.csv', 'MARUTI.csv', 'HEROMOTOCO.csv', 'NESTLEIND.csv', 'ULTRACEMCO.csv', 'HINDUNILVR.csv', 'IOC.csv', 'LT.csv', 'RELIANCE.csv', 'BPCL.csv', 'ADANIPORTS.csv', 'BAJFINANCE.csv', 'M&M.csv', 'INFY.csv', 'TCS.csv', 'NTPC.csv', 'HDFC.csv', 'JSWSTEEL.csv', 'HCLTECH.csv', 'POWERGRID.csv', 'SBIN.csv', 'SHREECEM.csv', 'TATASTEEL.csv', 'HINDALCO.csv', 'UPL.csv']
dict_keys(['ONGC', 'GAIL', 'MARUTI', 'HEROMOTOCO', 'NESTLEIND', 'ULTRACEMCO', 'HINDUNILVR', 'IOC', 'LT', 'RELIANCE', 'BPCL', 'ADANIPORTS', 'BAJFINANCE', 'M&M', 'INFY', 'TCS', 'NTPC', 'HDFC', 'JSWSTEEL', 'HCLTECH', 'POWERGRID', 'SBIN', 'SHREECEM', 'TATASTEEL', 'HINDALCO', 'UPL'])


In [7]:
#Weekly
def shift_returns(shift):
  days_to_shift = shift
  for i in db.keys():
    shifted_db = db[i].shift(days_to_shift)
    returns = shifted_db['adjusted_close'] - db[i]['adjusted_close']
    log_returns = np.log(shifted_db['adjusted_close'] / db[i]['adjusted_close'])
    db[i]['return'] = returns
    db[i]['log_return'] = log_returns


Momentum Based Trading System.
Choose the top and bottom performers and then long and short them respectively.

In [9]:
#Momentum Trading
shift_returns(5)
print(db['ONGC'].head(10))
table = []
for i in range(3000):
  signal = [0 for i in range(len(stocks))]
  order = [db[x]['log_return'][i] for x in stocks]
  order = pd.DataFrame(order).nlargest(3,0).index
  for x in order:
    signal[x] = 1
  table.append(signal)
table = pd.DataFrame(table , columns = stocks)
#print(table)

count = [table[stock].value_counts()[1] for stock in stocks]
count = pd.DataFrame(count)
long_momentum = [stocks[x] for x in count.nlargest(3,0).index]
short_momentum = [stocks[x] for x in count.nsmallest(3,0).index]

print("Rising stocks, Top stocks to long are  : " , long_momentum)
print("Dropping stocks, Top stocks to short are : ", short_momentum)

    timestamp    open    high  ...  split_coefficient  return  log_return
0  2021-03-05  116.25  118.20  ...                1.0     NaN         NaN
1  2021-03-04  113.80  117.05  ...                1.0     NaN         NaN
2  2021-03-03  114.45  115.90  ...                1.0     NaN         NaN
3  2021-03-02  116.00  116.65  ...                1.0     NaN         NaN
4  2021-03-01  114.40  117.50  ...                1.0     NaN         NaN
5  2021-02-26  115.90  118.40  ...                1.0    3.75    0.033182
6  2021-02-25  115.60  120.50  ...                1.0   -6.30   -0.054394
7  2021-02-24  113.80  115.50  ...                1.0    0.30    0.002636
8  2021-02-23  109.25  114.35  ...                1.0    1.25    0.011079
9  2021-02-22  105.55  108.60  ...                1.0   10.85    0.097190

[10 rows x 11 columns]
Rising stocks, Top stocks to long are  :  ['M&M', 'BAJFINANCE', 'SBIN']
Dropping stocks, Top stocks to short are :  ['NTPC', 'POWERGRID', 'HDFC']


Pair Trading: Look at 2 or more stocks with similar traits and use that data to predict movement

In [74]:
#Pair Trading using Cosine Similarity
j = 'ONGC'
days = 365
table = []
i_count = 0
j_count = 0
for i in stocks:
  table.append([])
  j_count = 0
  for j in stocks:
      if i == j:
        table[-1].append(1)
        continue
      if i_count > j_count:
        table[-1].append(table[j_count][i_count])
        continue
      data1 = db[i]
      data2 = db[j]
      list_of_tuples = list(zip(data1["timestamp"], data1["log_return"],data2["log_return"]))
      
      tmp = pd.DataFrame(list_of_tuples, columns = ["time" , i , j])
      #tmp.plot(x = "time" , y = [i,j])
      #print(tmp.head(10))
      numerator = tmp[i] * tmp[j]
      numerator = numerator.sum(skipna = True)
      deno = ((tmp[i] ** 2 ).sum(skipna = True))**0.5 * ((tmp[j]**2).sum(skipna = True))**0.5
      #print("NUMERATOR " , deno)
      #print("SIMILARITY INDEX OF " , i , " AND ", j , " IS " , numerator/deno)
      table[-1].append(numerator/deno)
      j_count+=1
i_count = 0
    
print("SIMILARITY TABLE")
table = pd.DataFrame(table,columns = stocks,index=stocks)
print(table)
for i in stocks:
  #print(table.nlargest(3,i))
  print("TOP 3 stocks like " , i , " are ",list(table.nlargest(4,i).index[1:]))

SIMILARITY TABLE
                ONGC      GAIL    MARUTI  ...  TATASTEEL  HINDALCO       UPL
ONGC        1.000000  0.557563  0.370124  ...   0.489011  0.492421  0.385197
GAIL        0.557563  1.000000  0.396415  ...   0.457186  0.474979  0.358778
MARUTI      0.370124  0.396415  1.000000  ...   0.430722  0.432183  0.384579
HEROMOTOCO  0.288636  0.324807  0.464606  ...   0.343541  0.357380  0.307150
NESTLEIND   0.213280  0.205329  0.262987  ...   0.242580  0.227454  0.220728
ULTRACEMCO  0.402046  0.401987  0.433561  ...   0.463686  0.470640  0.374320
HINDUNILVR  0.288419  0.241148  0.337238  ...   0.271905  0.262731  0.244040
IOC         0.459207  0.403568  0.342399  ...   0.331717  0.324477  0.308904
LT          0.490217  0.439515  0.487758  ...   0.557657  0.527017  0.467593
RELIANCE    0.494883  0.453746  0.397652  ...   0.534103  0.534819  0.425461
BPCL        0.425825  0.416101  0.357100  ...   0.281684  0.313225  0.312927
ADANIPORTS  0.373081  0.341226  0.382523  ...   0.477059  0

In [ ]:
'''
# BOLLINGER BANDS 
days = 365
gap = days//10
mean = []
upar = []
neeche = []
for i in range(days):
  low = i-gap
  if low<0:
    low = 0
  mean.append(data["adjusted_close"][low:i+gap].mean())
  upar.append(mean[-1] + data["adjusted_close"][low:i+5].std())
  neeche.append(mean[-1] - data["adjusted_close"][low:i+5].std())

list_of_tuples = list(zip(data["timestamp"][:days][::-1], data["adjusted_close"][:days][::-1], mean[::-1], upar[::-1] , neeche[::-1]))  
tmp = pd.DataFrame(list_of_tuples,columns=["time","close","mean",'upar','neeche']  )
tmp.plot(x = "time" , y = ["close","mean",'upar','neeche'])

#SIMILARITY / PAIR TRADING

j = 'ONGC'
days = 365
for i in db.keys():
    if i!=j:
      data1 = db[i]
      data2 = db[j]
      list_of_tuples = list(zip(data1["timestamp"][:days][::-1], data1["return"][:days][::-1],data2["return"][:days][::-1]))
      
      tmp = pd.DataFrame(list_of_tuples, columns = ["time" , i , j])
      tmp.plot(x = "time" , y = [i,j])
  '''''